# Quantization 101
## Image compression
### Quantization
$
q(x;s,z)=round\left(\frac{x}{s}+z\right),\quad \text{where }s,\ z \text{ are hyperparameter}\\
$
### Dequantization
$
x\approx\hat{x}=s\cdot(q(x)-z)\\
$
#### $s$ and $z$ are calculated by system of equation 
$
s = \frac{\beta-\alpha}{\beta_q-\alpha_q}\qquad
z = \frac{\beta\alpha_q-\alpha\beta_q}{\beta-\alpha}
$

In [1]:
import tensorflow as tf

#### Initiate weight

In [2]:
w = tf.keras.initializers.VarianceScaling(scale=2, mode='fan_in')(shape=[15,])
dtype = w.dtype
alpha = tf.math.reduce_min(w).numpy()
beta = tf.math.reduce_max(w).numpy()
print(f'type: {dtype}\nalpha: {alpha}\nbeta: {beta}')

type: <dtype: 'float32'>
alpha: -0.8072044849395752
beta: 0.45924633741378784


#### Calculate $s$ and $z$

In [11]:
def get_sz(X, bit_width=8):
    alpha = tf.math.reduce_min(w).numpy()
    beta = tf.math.reduce_max(w).numpy()
    alpha_q = 0
    beta_q = 2 ** bit_width -1
    s = (beta - alpha) / (beta_q - alpha_q)
    z = (beta * alpha_q - alpha * beta_q) / (beta - alpha)
    return s, z

In [12]:
s, z = get_sz(w)
print(f's: {s}\nz: {z}')

s: 0.0049664740468941485
z: 162.53069628831977


## Model quantization

$
q(x;s,z,b)=clip\left(round\left(\frac{x}{s}+z\right);0,2^{b}-1\right),\quad \text{where }s,\ z,\ b \text{ are hyperparameter}
$

$
clip(x;q_L,q_H)=
\begin{cases}
q_L,\quad x<q_L\\
x,\quad \ \ q_L\le x\le q_H\\
q_H,\quad q_H<x\\
\end{cases}
$

In [5]:
class Quantizer:
    def __init__(self, bit_width:int = 8):
        self.bit_width = bit_width
        self.step_size = None
        self.center = None

    def quantizer(self, X):
        X_min = tf.math.reduce_min(X)
        X_max = tf.math.reduce_max(X)
        Xq_min = 0
        Xq_max = tf.cast(tf.math.pow(2, self.bit_width) - 1, 'float32')
        self.step_size = (X_max - X_min) / (Xq_max - Xq_min)
        self.center = tf.math.round((X_max * Xq_min - X_min * Xq_max) / (X_max - X_min))
        print(self.step_size, self.center)
        Xq = tf.clip_by_value(tf.math.add(tf.math.divide(X, self.step_size),
                                          self.center), Xq_min, Xq_max)
        return tf.cast(Xq, 'uint8')

    def dequantizer(self, X):
        X = tf.cast(X, tf.float32)
        return self.step_size * (X - self.center)

In [6]:
quant = Quantizer()
q = quant.quantizer(w)
dq = quant.dequantizer(q)

tf.Tensor(0.004966474, shape=(), dtype=float32) tf.Tensor(163.0, shape=(), dtype=float32)


#### Quantization error

In [10]:
diff = w - dq
print(diff)

tf.Tensor(
[0.00233078 0.00200894 0.00316533 0.00284597 0.00184439 0.00357093
 0.00233072 0.00277051 0.00047642 0.00436997 0.00341664 0.00368279
 0.00462785 0.00264013 0.00195391], shape=(15,), dtype=float32)
